<a href="https://colab.research.google.com/github/paucaroscanoa/ApiBookAuthor/blob/master/Caso_de_Estudio_6_2_Dise%C3%B1o_de_predicciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Caso de Estudio 6.2 - Diseño de predicciones con datos sobre ventas en el Reino Unido

# Configuración

Ejecute (Run) estas celdas para instalar los paquetes necesarios para completar el caso de estudio. Esto podría llevar unos minutos así que sea paciente.

<h1 style="color:red;">ATENCIÓN: Puede que vea errores cuando ejecute las celdas siguientes. Sin embargo, no se preocupe, con tal de que pueda ejecutar la celda de importación de librerías (dos más abajo) y vea el mensaje "Librerías importadas con éxito!" es suficiente y puede continuar con el caso de estudio.<h1>

In [1]:
!pip install -q featuretools==1.29.0
import featuretools as ft
print('Librerías instaladas con éxito!')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.1/620.1 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.2/215.2 kB 5.4 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.10/dist-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


Librerías instaladas con éxito!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

<h1>Atención:</h1>

Ahora ha de reiniciar el entorno de ejecución. Para ello vaya a:

> Entorno de ejecución > _Reiniciar entorno de ejecución_

en la parte superior de su pantalla. Esto asegurará que sus cambio se han realizado con éxito.


# Importar

Sincronice su cuenta de Google. Para ello, siga el link que aparece en la salida de la siguiente celda una vez ejecutada. Copie el código que le aparece en pantalla e introdúzcalo en la salida de la celda. Una vez vea el mensaje: `Google Drive sincronizado con éxito!` puede continuar ejecutando el resto de celdas.

In [2]:
!mkdir uk-retail-data

In [3]:
from google.colab import auth
auth.authenticate_user()

from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

data = drive.CreateFile({'id':'1feQJdQqOYUaxeUcC7fj7s-ib3Wuqj4Hf'})
data.GetContentFile('uk-retail-data/customers.csv')
data = drive.CreateFile({'id':'16spEigNWUceuyB5uyaFyo5gW3YX5Fkko'})
data.GetContentFile('uk-retail-data/invoices.csv')
data = drive.CreateFile({'id':'1RlgJaKznx931rnp-vUpki_zHOtkq1eEy'})
data.GetContentFile('uk-retail-data/item_purchases.csv')
data = drive.CreateFile({'id':'1Iwf8zTmLzZyyuhJg3CLc_zW4ZtpoviAy'})
data.GetContentFile('uk-retail-data/items.csv')

utils_file = drive.CreateFile({'id':'1UbV2z7L5vonCz3KFLywGs4U-p2g5gQwy'})
utils_file.GetContentFile('utils.py')

print('Google Drive sincronizado con éxito!')

Google Drive sincronizado con éxito!


Importe las librerías necesarias para el desarrollo del caso.

In [4]:
import featuretools as ft
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import utils

from utils import (find_training_examples,
                   load_uk_retail_data,
                   engineer_features_uk_retail,
                   preview,
                   feature_importances)

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

print('Librerías importadas con éxito!')

Librerías importadas con éxito!


# Datos

Cargue la base de datos de ventas de diferentes productos en Reino Unido. Esto podría llevar unos minutos, así que sea paciente.

In [5]:
item_purchases, invoices, items, customers = load_uk_retail_data()
preview(items, 10)

,StockCode,Description,first_item_purchases_time
0,21730,GLASS STAR FROSTED T-LIGHT HOLDER,2010-12-01 08:26:00
1,22752,SET 7 BABUSHKA NESTING BOXES,2010-12-01 08:26:00
2,71053,WHITE METAL LANTERN,2010-12-01 08:26:00
3,84029E,RED WOOLLY HOTTIE WHITE HEART.,2010-12-01 08:26:00
4,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,2010-12-01 08:26:00
5,84406B,CREAM CUPID HEARTS COAT HANGER,2010-12-01 08:26:00
6,85123A,WHITE HANGING HEART T-LIGHT HOLDER,2010-12-01 08:26:00
7,22632,HAND WARMER RED POLKA DOT,2010-12-01 08:28:00
8,22633,HAND WARMER UNION JACK,2010-12-01 08:28:00
9,21754,HOME BUILDING BLOCK WORD,2010-12-01 08:34:00


In [6]:
item_purchases = item_purchases.dropna(subset="item_purchase_id")
items = items.dropna(subset="StockCode")
customers = customers.dropna(subset="CustomerID")
invoices = invoices.dropna(subset="InvoiceNo")

In [7]:
entities = {"item_purchases": (item_purchases,
                               "item_purchase_id",
                               "InvoiceDate"),
            "items": (items,
                      "StockCode"),
            "customers": (customers,
                          "CustomerID"),
            "invoices":(invoices,
                        "InvoiceNo",
                        "first_item_purchases_time")
            }
entities

{'item_purchases': (        item_purchase_id InvoiceNo StockCode  Quantity         InvoiceDate  \
  0                      0    536365    85123A         6 2010-12-01 08:26:00   
  1                      1    536365     71053         6 2010-12-01 08:26:00   
  2                      2    536365    84406B         8 2010-12-01 08:26:00   
  3                      3    536365    84029G         6 2010-12-01 08:26:00   
  4                      4    536365    84029E         6 2010-12-01 08:26:00   
  ...                  ...       ...       ...       ...                 ...   
  541904            541904    581587     22613        12 2011-12-09 12:50:00   
  541905            541905    581587     22899         6 2011-12-09 12:50:00   
  541906            541906    581587     23254         4 2011-12-09 12:50:00   
  541907            541907    581587     23255         4 2011-12-09 12:50:00   
  541908            541908    581587     22138         3 2011-12-09 12:50:00   
  
          UnitPrice

In [8]:
relationships = [("customers","CustomerID","invoices","CustomerID"),
                 ("invoices", "InvoiceNo","item_purchases", "InvoiceNo"),
                 ("items", "StockCode","item_purchases", "StockCode")]

## Encontrando ejemplos de entrenamiento

In [9]:
label_times = pd.DataFrame()

In [10]:
label_times = find_training_examples(item_purchases,
                                     invoices,
                                     prediction_window = pd.Timedelta("14d"),
                                     training_window = pd.Timedelta("21d"),
                                     lead = pd.Timedelta("7d"),
                                     threshold = 5)


In [11]:
label_times = label_times[['CustomerID','cutoff_time', 'purchases>threshold']].rename(columns={'cutoff_time': 'time', 'purchases>threshold': 'label'})

In [12]:
preview(label_times, 5)

,CustomerID,time,label
0,17505.0,2011-06-08,False
516,16444.0,2011-06-08,False
517,16889.0,2011-06-08,False
518,17613.0,2011-06-08,True
519,17152.0,2011-06-08,False


## Generación de variables

In [13]:
from featuretools.primitives import (Count, Day, Hour, Max, Mean, Median, Min,
                                     Minute, Month, Std, Sum, Week, Weekday,
                                     IsWeekend)

trans_primitives = [Minute, Hour, Day, Week, Month, Weekday, IsWeekend]
agg_primitives = [Mean, Max, Std]
training_window='21d'

feature_matrix, features = ft.dfs(dataframes=entities,
                                  relationships=relationships,
                                  target_dataframe_name="customers",
                                  trans_primitives=trans_primitives,
                                  agg_primitives=agg_primitives,
                                  cutoff_time=label_times,
                                  cutoff_time_in_index=True,
                                  training_window=training_window)
feature_matrix.drop("Country", axis=1, inplace=True)
feature_matrix = feature_matrix.sort_index()

/usr/local/lib/python3.10/dist-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/usr/local/lib/python3.10/dist-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/usr/local/lib/python3.10/dist-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/usr/local/lib/python3.10/dist-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. 

In [14]:
preview(feature_matrix, 10)

,,MAX(item_purchases.Quantity),MAX(item_purchases.UnitPrice),MEAN(item_purchases.Quantity),MEAN(item_purchases.UnitPrice),STD(item_purchases.Quantity),STD(item_purchases.UnitPrice),DAY(first_invoices_time),HOUR(first_invoices_time),IS_WEEKEND(first_invoices_time),MINUTE(first_invoices_time),...,MEAN(invoices.MAX(item_purchases.UnitPrice)),MEAN(invoices.MEAN(item_purchases.Quantity)),MEAN(invoices.MEAN(item_purchases.UnitPrice)),MEAN(invoices.STD(item_purchases.Quantity)),MEAN(invoices.STD(item_purchases.UnitPrice)),STD(invoices.MAX(item_purchases.Quantity)),STD(invoices.MAX(item_purchases.UnitPrice)),STD(invoices.MEAN(item_purchases.Quantity)),STD(invoices.MEAN(item_purchases.UnitPrice)),label
CustomerID,time,,,,,,,,,,,,,,,,,,,,,
12423.0,2011-06-08,24.0,15.00,8.074074,3.707407,4.999430,4.368663,21,10,False,54,...,15.000000,9.359091,3.767909,6.040492,5.132343,8.485281,0.000000,2.886281,0.135893,False
12431.0,2011-06-08,84.0,8.95,10.850000,4.032000,19.746485,3.032795,1,10,False,3,...,6.516667,1.196078,3.540000,11.529269,4.314799,50.846173,4.214657,10.778412,1.688579,False
12437.0,2011-06-08,48.0,18.00,16.000000,5.240000,14.000000,5.824917,12,14,False,13,...,18.000000,15.972727,5.243182,14.308902,5.946851,5.656854,0.000000,0.809959,0.094495,True
12464.0,2011-06-08,6.0,16.95,3.000000,11.408333,1.549193,5.825926,3,13,False,30,...,12.725000,2.750000,10.681250,0.866025,3.468091,2.828427,5.975052,1.060660,3.084753,False
12473.0,2011-06-08,48.0,18.00,11.255814,2.638605,15.485928,4.366770,1,10,False,15,...,12.983333,1.257143,7.197143,11.605644,2.463529,28.583212,8.689122,13.330432,9.382885,False
12474.0,2011-06-08,12.0,4.95,-0.300000,2.446000,4.498148,1.384985,6,12,False,0,...,3.516667,2.916667,1.986167,1.116025,1.403951,7.505553,2.482606,7.875331,1.159797,True
12481.0,2011-06-08,25.0,18.00,11.795455,2.619545,6.356025,3.365033,9,10,False,13,...,9.975000,12.225869,2.213745,5.816336,2.009097,0.707107,11.349064,0.892756,0.841703,False
12493.0,2011-06-08,12.0,18.00,6.692308,6.000000,4.404252,6.609778,26,14,False,53,...,17.475000,3.166667,11.018750,3.915780,5.987681,9.192388,0.742462,5.892557,8.388054,True
12540.0,2011-06-08,40.0,28.00,11.157143,4.252000,8.393641,5.553142,13,9,False,39,...,28.000000,10.228955,4.594061,7.771838,5.764126,10.606602,0.000000,3.828581,1.410930,False


## Entrenamiento de un modelo con *Random Forest*

In [15]:
X_y = feature_matrix
y = X_y.pop('label')
X_train, X_test, y_train, y_test = train_test_split(feature_matrix,
                                                    y,
                                                    test_size=0.35)

In [16]:
imp = SimpleImputer(missing_values=np.nan,
                    strategy='mean')
imp = imp.fit(X_train)
X_train_imp = imp.transform(X_train)

In [17]:
clf = RandomForestClassifier(random_state=0,
                             n_estimators=100,
                             class_weight="balanced",
                             verbose=True)
clf.fit(X_train_imp, y_train)

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.1s


RandomForestClassifier(class_weight='balanced', random_state=0, verbose=True)

## Prueba del modelo

In [18]:
X_test_imp = imp.transform(X_test)
predicted_labels = clf.predict(X_test_imp)

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s


In [19]:
tn, fp, fn, tp = confusion_matrix(y_test, predicted_labels).ravel()
tn, fp, fn, tp

(223, 3, 42, 7)

In [20]:
feature_importances(clf,feature_matrix.columns, n=15)

1: Feature: STD(invoices.MAX(item_purchases.UnitPrice)), 0.065
2: Feature: WEEK(first_invoices_time), 0.059
3: Feature: MEAN(item_purchases.UnitPrice), 0.046
4: Feature: STD(invoices.MEAN(item_purchases.UnitPrice)), 0.045
5: Feature: STD(item_purchases.UnitPrice), 0.045
6: Feature: MINUTE(first_invoices_time), 0.044
7: Feature: MAX(invoices.STD(item_purchases.UnitPrice)), 0.043
8: Feature: MEAN(invoices.STD(item_purchases.UnitPrice)), 0.043
9: Feature: MEAN(invoices.MEAN(item_purchases.Quantity)), 0.043
10: Feature: MEAN(invoices.MEAN(item_purchases.UnitPrice)), 0.041
11: Feature: MEAN(item_purchases.Quantity), 0.040
12: Feature: DAY(first_invoices_time), 0.040
13: Feature: MEAN(invoices.MAX(item_purchases.UnitPrice)), 0.038
14: Feature: MEAN(invoices.STD(item_purchases.Quantity)), 0.036
15: Feature: STD(invoices.MEAN(item_purchases.Quantity)), 0.035
